In [15]:
import MetaTrader5 as mt5
import pandas as pd
import datetime
import sys
import os
import mplfinance as mpf
import talib as ta
import numpy as np
import pandas_ta as ta

import warnings
warnings.filterwarnings('ignore')

from backtesting import Strategy, Backtest
from backtesting.test import SMA, GOOG
from backtesting.lib import crossover, TrailingStrategy

current = os.path.abspath('')
parent = os.path.dirname(current)
sys.path.append(parent)

from trade_utils import get_symbol_data, get_MT5_Timeframe

if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    
symbol = "SP500"

data = get_symbol_data(symbol, closed_candles_only=True, periods=1000, timeframe="5M")

data.rename(columns={"open": "Open", "close": "Close", "low": "Low", "high": "High", "volume": "Volume"}, inplace=True)

data.sort_index(ascending=True, inplace=True)

# drop all na's from our indicators, patterns and signal columns
data.dropna(subset=["signal_rsi_sell", "signal_rsi_buy", "ind_atr","ptn_bullish_engulfing_1", "ptn_bearish_engulfing_1"], inplace=True)

get symbol history


In [12]:


class SignalStrategy(Strategy):

    def init(self): 
        # before strategy is run
        # calculate indicators and other things used by strategy
        super().init()
        pass
    
    def next(self):
        # each candlestick
        super().next()

        # get latest value
        #print(self.data.signal_rsi_sell[-1])

        if not self.position:            
            if self.data.ptn_bullish_engulfing_1[-1] == True:
                stop_loss = self.data.Close[-1] - self.data.ind_atr[-1] * 1.5 
                self.buy(sl=stop_loss)
            elif self.data.ptn_bearish_engulfing_1[-1] == True:
                stop_loss = self.data.Close[-1] + self.data.ind_atr[-1] * 1.5
                self.sell(sl=stop_loss)
                

    

bt = Backtest(data, SignalStrategy, cash=50_000)
stats = bt.run()
print(stats)

Start                     2024-02-22 22:20:00
End                       2024-02-28 12:05:00
Duration                      5 days 13:45:00
Exposure Time [%]                   96.243655
Equity Final [$]                  50205.59336
Equity Peak [$]                   50391.80336
Return [%]                           0.411187
Buy & Hold Return [%]               -0.305405
Return (Ann.) [%]                   18.808686
Volatility (Ann.) [%]                3.200188
Sharpe Ratio                          5.87737
Sortino Ratio                        14.36876
Calmar Ratio                        45.272761
Max. Drawdown [%]                   -0.415453
Avg. Drawdown [%]                   -0.115013
Max. Drawdown Duration        2 days 08:25:00
Avg. Drawdown Duration        0 days 05:39:00
# Trades                                    5
Win Rate [%]                             20.0
Best Trade [%]                       0.641987
Worst Trade [%]                     -0.086311
Avg. Trade [%]                    

In [47]:

def indicator(data):
    # .s returns pandas series vs numpy array
    bbands = ta.bbands(close = data.Close.s, std=1)
    return bbands.to_numpy().T

class BBStrategy(Strategy):

    def init(self): 
        # before strategy is run
        # calculate indicators and other things used by strategy
        super().init()
        self.bbands = self.I(indicator, self.data)
        
        
        
    def next(self):
        # each candlestick
        super().next()

        # get latest value
        #print(self.data.signal_rsi_sell[-1])
        lower_band = self.bbands[0]
        mid_band = self.bbands[1]
        upper_band = self.bbands[2]

        print(upper_band[-1], mid_band[-1], upper_band[-1])
        
        if not self.position:
            if self.data.Close[-1] > upper_band[-1]:
                stop_loss = self.data.Close[-1] - self.data.ind_atr[-1] * 1.5 
                self.buy(sl=stop_loss)
            elif self.data.Close[-1] < lower_band[-1]:
                stop_loss = self.data.Close[-1] + self.data.ind_atr[-1] * 1.5 
                self.sell(sl=stop_loss)


    

bt = Backtest(data, BBStrategy, cash=50_000)
stats = bt.run()
print(stats)

5089.224118628493 5088.87 5089.224118628493
5089.097275117844 5088.749999999999 5089.097275117844
5088.72000002303 5088.539999999999 5088.72000002303
5088.752480790579 5088.589999999999 5088.752480790579
5088.962763641271 5088.69 5088.962763641271
5089.167359255477 5088.789999999999 5089.167359255477
5089.35199502165 5088.95 5089.35199502165
5089.541058901645 5089.21 5089.541058901645
5089.5099999688755 5089.35 5089.5099999688755
5089.51000000673 5089.389999999999 5089.51000000673
5089.862409995945 5089.549999999999 5089.862409995945
5089.955941178842 5089.65 5089.955941178842
5090.19792157534 5089.789999999999 5090.19792157534
5090.204400917262 5089.849999999999 5090.204400917262
5090.258193479753 5090.01 5090.258193479753
5090.729230199654 5090.21 5090.729230199654
5090.793321836934 5090.31 5090.793321836934
5090.936877589288 5090.410000000001 5090.936877589288
5091.121825195068 5090.710000000001 5091.121825195068
5091.153587141203 5090.65 5091.153587141203
5090.920613706427 5090.35 

In [39]:
bt.plot()

GridPlot(id='p1959', ...)

In [40]:
stats._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-9,8,12,5088.63,5089.630000,-9.000000,-0.000197,2024-02-22 23:10:00,2024-02-22 23:30:00,0 days 00:20:00
1,9,13,37,5089.73,5088.700309,-9.267222,-0.000202,2024-02-22 23:35:00,2024-02-23 01:35:00,0 days 02:00:00
2,-9,38,41,5088.13,5089.633497,-13.531473,-0.000295,2024-02-23 01:40:00,2024-02-23 01:55:00,0 days 00:15:00
3,9,42,42,5088.83,5087.777743,-9.470312,-0.000207,2024-02-23 02:00:00,2024-02-23 02:00:00,0 days 00:00:00
4,-9,43,52,5087.50,5088.837453,-12.037076,-0.000263,2024-02-23 02:05:00,2024-02-23 02:50:00,0 days 00:45:00
5,9,53,54,5089.28,5087.521247,-15.828780,-0.000346,2024-02-23 02:55:00,2024-02-23 03:00:00,0 days 00:05:00
6,-9,55,107,5086.90,5088.537777,-14.739994,-0.000322,2024-02-23 03:05:00,2024-02-23 07:25:00,0 days 04:20:00
7,9,108,150,5088.98,5086.722667,-20.315999,-0.000444,2024-02-23 07:30:00,2024-02-23 11:00:00,0 days 03:30:00
8,-9,151,157,5085.68,5092.816131,-64.225177,-0.001403,2024-02-23 11:05:00,2024-02-23 11:35:00,0 days 00:30:00
9,9,158,191,5093.19,5085.961882,-65.053062,-0.001419,2024-02-23 11:40:00,2024-02-23 14:25:00,0 days 02:45:00
